In [1]:
import numpy as np
import cv2, PIL
from cv2 import aruco
from scipy.spatial import distance as dist
import imutils
from imutils import perspective
from imutils import contours
import glob
import pickle
import time
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# import pandas as pd
# import math
# import serial
# %matplotlib nbagg


In [2]:
def find_aruco_marker(img,aruco_dictionary,aruco_parameters):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Lists of ids and the corners belonging to each id
    corners, ids, rejected_points = aruco.detectMarkers(gray, aruco_dictionary, parameters=aruco_parameters)

    return corners,ids

In [3]:
def find_pts(corners,img_marker,ids):
    pts = np.empty((4,2),dtype=np.float32)
    for i in range(len(ids)):
        pt_x,pt_y = corners[i][:, 0][i], corners[i][:, 1][i]
        cv2.circle(img_marker, (int(pt_x),int(pt_y)), 1, (0, 0, 255), -1)
        pts[i][0] = pt_x
        pts[i][1] = pt_y
        
    return pts,img_marker

In [4]:
def four_point_transform(image, pts,t,MeasuredWidth,MeasuredHeight):
    height, width, _ = image.shape
    R = MeasuredWidth/MeasuredHeight
#     print("width,height =",width,height)
    rect = pts
    (tl, tr, br, bl) = rect
#     print("rect=",rect)

    width = height*R
#     height = width / R
    dst = np.array([
        [0+t,0+t],
        [width+t, 0+t],
        [width+t, height+t],
        [0+t,height+t]], dtype="float32")
#     print("dst =",dst)

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    #print(M)
    # Apply perspective transform to the image
    warped = cv2.warpPerspective(image, M, (int(width+2*t),int(height+2*t)))
    
    x_org , y_org = 0+t, height+t
    
    return warped, dst , x_org , y_org 


In [5]:
def calculate_ppm(MeasuredWidth,MeasuredHeight,printSize,corners,ids,flag):
    pixelsPerMetric = np.empty((4,1))
    if flag == "rec":
        pixelsPerMetric[0] = dist.euclidean(corners[0][0], corners[1][1])/MeasuredWidth
        pixelsPerMetric[1] = dist.euclidean(corners[1][1], corners[2][2])/MeasuredHeight 
        pixelsPerMetric[2]= dist.euclidean(corners[2][2], corners[3][3])/MeasuredWidth 
        pixelsPerMetric[3] = dist.euclidean(corners[3][3], corners[0][0])/MeasuredHeight
        
        
    else:
        for i in range(len(ids)):
            distance_top = dist.euclidean(corners[i][0], corners[i][1]) 
            distance_right = dist.euclidean(corners[i][1], corners[i][2]) 
            distance_bottom = dist.euclidean(corners[i][2], corners[i][3]) 
            distance_left = dist.euclidean(corners[i][3], corners[i][0])

            pixelsPerMetric[i] = (distance_top+distance_right+distance_bottom+distance_left) /(4*printSize)
   
    return pixelsPerMetric.mean()

In [6]:
def draw_rect(img):
    for i in range(len(ids)):
            if i!=3:
                start_point = (int((corners[i][i])[0]),int((corners[i][i])[1]))
                end_point = (int((corners[i+1][i+1])[0]),int((corners[i+1][i+1])[1]))
            elif i==3:
                start_point = (int(tuple(corners[i][i])[0]),int((corners[i][i])[1]))
                end_point = (int((corners[0][0])[0]),int((corners[0][0])[1]))
            cv2.line(img, start_point, end_point, (0,255,0), 1)
            
    # # Define the rectangle parameters
    # start_point = (int(tuple(corners[0][0])[0]),int(tuple(corners[0][0])[1]))
    # end_point = (int(tuple(corners[2][2])[0]),int(tuple(corners[2][2])[1]))
    # color = (0, 0, 255) # Green
    # # Draw the rectangle on the image
    # # cv2.rectangle(img_obj_marker, start_point, end_point, color, 1)
    
    return img

In [7]:
def color_detection(trans_image,obj_lower, obj_upper):
    center_x, center_y = 0, 0
    # Convert the frame from BGR color space to HSV color space
    hsv = cv2.cvtColor(trans_image, cv2.COLOR_BGR2HSV)

    # Threshold the image to get the green color region
    mask = cv2.inRange(hsv, obj_lower, obj_upper)

    # Perform morphological operations to remove noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours in the thresholded image
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        # Get the contour with the largest area
        max_contour = max(contours, key=cv2.contourArea)
        
        # Get the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(max_contour)

        # Draw a green rectangle around the object
        cv2.rectangle(trans_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Get the center of the bounding rectangle
        center_x, center_y = x + w // 2, y + h // 2

                
    return trans_image,center_x, center_y

# Main

In [8]:
aruco_dict_corner= aruco.Dictionary_get(aruco.DICT_4X4_250)
parameters_corner= aruco.DetectorParameters_create()
aruco_dict_obj= aruco.Dictionary_get(aruco.DICT_6X6_250)
parameters_obj= aruco.DetectorParameters_create()
MeasuredWidth,MeasuredHeight =151.2 , 86.3
printSize = 6.1
t = 60
r = 1
num = 1
flag = False
detector_list_marker = []
detector_list_color = []
id_num1 = 1
id_num2 = 4
obj_lower = (0,128,29)
obj_upper = (179, 255, 255)


# Define the kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))

# Define the initial position and center of the object
x, y, w, h = 0, 0, 0, 0
center_x, center_y, prev_center_y, prev_center_x = 0, 0, 0, 0

# Open the video file
cap = cv2.VideoCapture(0)
# time.sleep(2)

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()
    
while True:
    # Read a frame from the video
    ret, img = cap.read()
#     img = cv2.imread("./pictures/four_marker_11.jpg")
    
#      If the frame cannot be read, break out of the loop
    if not ret:
        break
        
    # img = cv2.imread("./pictures/four_marker_6.jpg")
#     cv2.imwrite("./pictures/webcam.jpg",img)
    height, width, _ = img.shape

    img_resized = cv2.resize(img,(int(width/r), int(height/r)))
    # cv2.imshow("Image_resized", img_resized)

    corners,ids = find_aruco_marker(img_resized,aruco_dict_corner,parameters_corner)
    
    if ids is not None:
        if (len(ids)>1):
            # Sort the IDs and corners in ascending order
            ids = np.squeeze(ids)
            corners = np.squeeze(corners)
            sort_index = np.argsort(ids)
            ids = ids[sort_index]
            corners = corners[sort_index]
    
    temp = np.copy(corners[3])
    corners[3] = corners [2]
    corners [2] = temp

#     Print the sorted IDs and corners
    print("Sorted IDs:", ids)
#     print("Sorted corners:", corners)

    pts,img_marker = find_pts(corners,img_resized,ids)
#     cv2.imshow("Detected_markers&centers",img_marker)

    img_trans,dst,x_org , y_org = four_point_transform(img_resized,pts,t,MeasuredWidth,MeasuredHeight)

#     cv2.imshow("Transformed",img_trans)
    # cv2.imwrite("./pictures/trans1.jpg",img_trans)

    corners,ids = find_aruco_marker(img_trans,aruco_dict_corner,parameters_corner)
    
    if ids is not None:
        if (len(ids)>1):
            # Sort the IDs and corners in ascending order
            ids = np.squeeze(ids)
            corners = np.squeeze(corners)
            sort_index = np.argsort(ids)
            ids = ids[sort_index]
            corners = corners[sort_index]

    temp = np.copy(corners[3])
    corners[3] = corners [2]
    corners [2] = temp

    img_trans = draw_rect(img_trans)
    

    PPM1= calculate_ppm(MeasuredWidth,MeasuredHeight,printSize,corners,ids,flag="")
    PPM2= calculate_ppm(MeasuredWidth,MeasuredHeight,printSize,corners,ids,flag="rec")
    PPM = PPM2
    print("PPM1 =",PPM1)
    print("PPM2 =",PPM2)

    #------------------------ Loop through the frames of the video --------------------
    while True:
        # Read a frame from the video
        ret, img = cap.read()
#         img = cv2.imread("./pictures/four_marker_11.jpg")

        #If the frame cannot be read, break out of the loop
#         if not ret:
#             break
            
        height, width, _ = img.shape

        img_resized = cv2.resize(img,(int(width/r), int(height/r)))
        
        img_trans,dst,x_org , y_org = four_point_transform(img,pts,t,MeasuredWidth,MeasuredHeight)
        
        img_trans,center_x, center_y=color_detection(img_trans,obj_lower, obj_upper)
        
        corners_obj,ids_obj = find_aruco_marker(img_trans,aruco_dict_obj,parameters_obj)
        if ids_obj is None:
            cv2.imshow("Detectected_object", img_trans)
            continue
            
        if (len(ids_obj)<1):
            cv2.imshow("Detectected_object", img_trans)
            continue
        
        # Draw what the aruco detector sees
        img_obj_marker = aruco.drawDetectedMarkers(img_trans.copy(),corners_obj,ids_obj)    
        
#         if (len(ids_obj)>0):
#             # Sort the IDs and corners in ascending order
#             ids_obj = np.squeeze(ids_obj)
#             corners_obj = np.squeeze(corners_obj)
#             sort_index = np.argsort(ids_obj)
#             ids_obj = ids_obj[sort_index]
#             corners_obj = corners_obj[sort_index]
        
        indices_obj1 = np.where(ids_obj==id_num1)[0]
        indices_obj2 = np.where(ids_obj==id_num2)[0]
        if len(indices_obj1) > 0 and len(indices_obj2) > 0  :
            x_obj1,y_obj1 = corners_obj[0][0][:, 0].mean(), corners_obj[0][0][:, 1].mean()
            cv2.circle(img_obj_marker, (int(x_obj1),int(y_obj1)), 2, (0, 0, 255), -1)
            x_obj2,y_obj2 = corners_obj[1][0][:, 0].mean(), corners_obj[1][0][:, 1].mean()
            cv2.circle(img_obj_marker, (int(x_obj2),int(y_obj2)), 2, (0, 0, 255), -1)
        else:
#             print("Error finding object marker!")
            cv2.imshow("Detectected_object",img_obj_marker)
#             time.sleep(3)
            continue
        
        x_shift , y_shift = +6.5 , 6.2
        
        x_coordinate1, y_coordinate1 = (x_obj1 - x_org)/PPM + x_shift ,-(y_obj1 - y_org)/PPM + y_shift
        x_coordinate2, y_coordinate2 = (x_obj2 - x_org)/PPM + x_shift  ,-(y_obj2 - y_org)/PPM + y_shift
#         rounded_x1, rounded_y1= round(x_coordinate1, 2),round(y_coordinate1, 2)
#         formatted_x1, formatted_y1 = '{:.2f}'.format(rounded_x1),'{:.2f}'.format(rounded_y1)
#         cv2.putText(img_obj_marker, f'id1 = ({formatted_x1}, {formatted_y1})', (400,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 1)
#         rounded_x2, rounded_y2= round(x_coordinate2, 2),round(y_coordinate2, 2)
#         formatted_x2, formatted_y2 = '{:.2f}'.format(rounded_x2),'{:.2f}'.format(rounded_y2)
#         cv2.putText(img_obj_marker, f'id4 = ({formatted_x2}, {formatted_y2})', (400,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 1)
        
        x_center_marker, y_center_marker = (abs(x_coordinate2 - x_coordinate1)/2)+min(x_coordinate1,x_coordinate2) , (abs(y_coordinate2 - y_coordinate1)/2)+min(y_coordinate1,y_coordinate2)
        rounded_x, rounded_y= round(x_center_marker, 2),round(y_center_marker, 2)
        formatted_x, formatted_y = '{:.2f}'.format(rounded_x),'{:.2f}'.format(rounded_y)
        cv2.putText(img_obj_marker, f'marker= ({formatted_x}, {formatted_y})', (50,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (138,43,226), 1)
        
        x_center_color, y_center_color = (center_x - x_org)/PPM + x_shift, -(center_y - y_org)/PPM + y_shift
        rounded_x_color, rounded_y_color= round(x_center_color, 2),round(y_center_color, 2)
        formatted_x_color, formatted_y_color = '{:.2f}'.format(rounded_x_color),'{:.2f}'.format(rounded_y_color)
        cv2.putText(img_obj_marker, f'color= ({formatted_x_color}, {formatted_y_color})', (500,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
        
        cv2.circle(img_obj_marker, (int(abs((x_obj1-x_obj2))/2 + min(x_obj1,x_obj2)),int(abs((y_obj1-y_obj2))/2 + min(y_obj1,y_obj2))), 2, (238,130,238), -1)
        cv2.circle(img_obj_marker, (center_x,center_y), 2, (0,255,0), -1)
#         img_obj_marker = draw_rect(img_obj_marker)
        
        cv2.imshow("Detectected_object",img_obj_marker)

        if cv2.waitKey(1) == ord('o'):
            print("PPM changed.")
            break
            
        if cv2.waitKey(1) == ord('p'):
            print ("num =",num)
            print("detected values using marker: ")
            print( x_center_marker,",", y_center_marker)
            detector_list_marker.append(x_center_marker)
            detector_list_marker.append(y_center_marker)
            
            
            print("detected values using color: ")
            print( x_center_color, ",",y_center_color)
            detector_list_color.append(x_center_color)
            detector_list_color.append(y_center_color)
            
            cv2.imwrite('./SavedImages/img' + str(num) + '.jpg', img)
            cv2.imwrite('./SavedImages/detected/img' + str(num) + '.jpg', img_obj_marker)
            print("image saved!")
            num += 1
            time.sleep(3)
            

        if cv2.waitKey(1) == ord('q'):
            # Set the flag to True and exit the inner loop
            flag = True
            break 

    if flag:
        # Exit the outer loop when the flag is True
        break
    # Release the video capture object and close all windows
cap.release()   
cv2.destroyAllWindows()

Sorted IDs: [1 3 4 6]
PPM1 = 5.467953212925646
PPM2 = 5.574923984143773
num = 1
detected values using marker: 
17.262478586372175 , 24.002933328290638
detected values using color: 
17.800602515690784 , 24.496213596832696
image saved!
num = 2
detected values using marker: 
34.59455347650903 , 5.684297901069667
detected values using color: 
35.19994289699247 , 6.020625356893797
image saved!
num = 3
detected values using marker: 
101.25465522085167 , 33.59947673447249
detected values using color: 
101.20981156007514 , 34.54119361078006
image saved!
num = 4
detected values using marker: 
80.13329099509629 , 9.765071031735783
detected values using color: 
80.40235295975559 , 10.684366077655072
image saved!
num = 5
detected values using marker: 
92.42045404787119 , 29.272063469535347
detected values using color: 
92.42045404787119 , 30.056827533124984
image saved!
num = 6
detected values using marker: 
106.34441071899019 , 25.751836098576113
detected values using color: 
106.05292692394261 ,

In [16]:
f1 = open("cam1_marker.txt", "w")
f2 = open("cam1_color.txt", "w")

for i in range(len(detector_list_marker)):
    f1.write(str(detector_list_marker[i]) + ',')
    
for i in range(len(detector_list_color)):
    f2.write(str(detector_list_color[i]) + ',')


f1.close()
f2.close()

In [10]:
ids

array([1, 3, 4, 6], dtype=int32)

In [11]:
corners_obj[0][0][:, 0]

array([228., 253., 247., 222.], dtype=float32)

In [12]:
corners_obj

(array([[[228.,  93.],
         [253.,  98.],
         [247., 120.],
         [222., 116.]]], dtype=float32),
 array([[[116.,  73.],
         [141.,  77.],
         [134., 100.],
         [109.,  95.]]], dtype=float32))

In [13]:
ids_obj

array([[4],
       [1]], dtype=int32)

In [14]:
import cv2
img = cv2.imread("./pictures/four_marker_11.jpg")
    
     #If the frame cannot be read, break out of the loop
#     if not ret:
#         break
        
    # img = cv2.imread("./pictures/four_marker_6.jpg")
#     cv2.imwrite("./pictures/webcam.jpg",img)
height, width, _ = img.shape

In [15]:
print(height, width)

540 720
